--- Day 13: Distress Signal ---

You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a distress signal.

Your handheld device must still not be working properly; the packets from the distress signal got decoded out of order. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify how many pairs of packets are in the right order.

For example:
```
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
```
Packet data consists of lists and integers. Each list starts with [, ends with ], and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called left and the second value is called right. Then:

    If both values are integers, the lower integer should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
    If both values are lists, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
    If exactly one value is an integer, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing [0,0,0] and 2, convert the right value to [2] (a list containing 2); the result is then found by instead comparing [0,0,0] and [2].

Using these rules, you can determine which of the pairs in the example are in the right order:
```
== Pair 1 ==
- Compare [1,1,3,1,1] vs [1,1,5,1,1]
  - Compare 1 vs 1
  - Compare 1 vs 1
  - Compare 3 vs 5
    - Left side is smaller, so inputs are in the right order

== Pair 2 ==
- Compare [[1],[2,3,4]] vs [[1],4]
  - Compare [1] vs [1]
    - Compare 1 vs 1
  - Compare [2,3,4] vs 4
    - Mixed types; convert right to [4] and retry comparison
    - Compare [2,3,4] vs [4]
      - Compare 2 vs 4
        - Left side is smaller, so inputs are in the right order

== Pair 3 ==
- Compare [9] vs [[8,7,6]]
  - Compare 9 vs [8,7,6]
    - Mixed types; convert left to [9] and retry comparison
    - Compare [9] vs [8,7,6]
      - Compare 9 vs 8
        - Right side is smaller, so inputs are not in the right order

== Pair 4 ==
- Compare [[4,4],4,4] vs [[4,4],4,4,4]
  - Compare [4,4] vs [4,4]
    - Compare 4 vs 4
    - Compare 4 vs 4
  - Compare 4 vs 4
  - Compare 4 vs 4
  - Left side ran out of items, so inputs are in the right order

== Pair 5 ==
- Compare [7,7,7,7] vs [7,7,7]
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Right side ran out of items, so inputs are not in the right order

== Pair 6 ==
- Compare [] vs [3]
  - Left side ran out of items, so inputs are in the right order

== Pair 7 ==
- Compare [[[]]] vs [[]]
  - Compare [[]] vs []
    - Right side ran out of items, so inputs are not in the right order

== Pair 8 ==
- Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
  - Compare 1 vs 1
  - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
    - Compare 2 vs 2
    - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
      - Compare 3 vs 3
      - Compare [4,[5,6,7]] vs [4,[5,6,0]]
        - Compare 4 vs 4
        - Compare [5,6,7] vs [5,6,0]
          - Compare 5 vs 5
          - Compare 6 vs 6
          - Compare 7 vs 0
            - Right side is smaller, so inputs are not in the right order
```
What are the indices of the pairs that are already in the right order? (The first pair has index 1, the second pair has index 2, and so on.) In the above example, the pairs in the right order are 1, 2, 4, and 6; the sum of these indices is 13.

Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?


In [106]:
// Imports
#r "nuget:Xunit,2.4.1"

Installed Packages Xunit, 2.4.1

In [107]:
// Data Types and Functions
public class Node {
    public List<Node> Children { get; set; } = new List<Node>();
    public Node Parent { get; set; }
    public int? Value { get; set; }

    public override string ToString() {
        if (Value != null) {
            return $"{Value}";
        } else {
            return "[" + String.Join(",", Children.Select(c => c.ToString())) + "]";
        }
    }
}

public Node ParseNode(string input) {
    var currentNode = new Node();
    int? currentValue = null;
    foreach (var c in input) {
        if (c == '[') {
            var newNode = new Node();
            newNode.Parent = currentNode;
            currentNode.Children.Add(newNode);
            currentNode = newNode;
            continue;
        }

        if (char.IsAsciiDigit(c)) {
            currentValue = (currentValue ?? 0) * 10 + int.Parse($"{c}");
            continue;
        }

        if (c == ',') {
            if (currentValue != null) {
                var newNode = new Node();
                newNode.Value = currentValue;
                newNode.Parent = currentNode;
                currentNode.Children.Add(newNode);
            }
            currentValue = null;
            continue;
        }

        if (c == ']') {
            if (currentValue != null) {
                var newNode = new Node();
                newNode.Value = currentValue;
                newNode.Parent = currentNode;
                currentNode.Children.Add(newNode);
            }
            currentValue = null;

            currentNode = currentNode.Parent;
            continue;
        }
    }
    return currentNode;
}

public enum Order {
    LessThan = -1,
    Equal = 0,
    GreaterThan = 1,
}

public static Order GetOrder(Node a, Node b) {
    // Console.WriteLine($"{a} vs {b}");
    if (a.Value != null && b.Value != null) {
        if (a.Value < b.Value) {
            return Order.LessThan;
        } else if (a.Value == b.Value) {
            return Order.Equal;
        } else {
            return Order.GreaterThan;
        }
    }

    if (a.Value != null) {
        var aAsList = new Node();
        aAsList.Children = new List<Node>();
        aAsList.Children.Add(a);
        return GetOrder(aAsList, b);
    }

    if (b.Value != null) {
        var bAsList = new Node();
        bAsList.Children = new List<Node>();
        bAsList.Children.Add(b);
        return GetOrder(a, bAsList);
    }

    foreach ((var aItem, var bItem) in a.Children.Zip(b.Children)) {
        var itemOrder = GetOrder(aItem, bItem);
        if (itemOrder != Order.Equal) {
            return itemOrder;
        }
    }

    if (a.Children.Count < b.Children.Count) {
        return Order.LessThan;
    } else if (a.Children.Count == b.Children.Count) {
        return Order.Equal;
    } else {
        return Order.GreaterThan;
    }
}

class NodeComparer : IComparer<Node>
{
    public int Compare(Node a, Node b)
    {
      return (int) GetOrder(a, b);
    }
}

In [108]:
// Example tests
using Xunit;

Assert.Equal(Order.LessThan, GetOrder(ParseNode("[1,1,3,1,1]"), ParseNode("[1,1,5,1,1]")));
Assert.Equal(Order.LessThan, GetOrder(ParseNode("[[1],[2,3,4]]"), ParseNode("[[1],4]")));
Assert.Equal(Order.GreaterThan, GetOrder(ParseNode("[9]"), ParseNode("[[8,7,6]]")));
Assert.Equal(Order.LessThan, GetOrder(ParseNode("[[[4,4],4,4]"), ParseNode("[[4,4],4,4,4]")));
Assert.Equal(Order.GreaterThan, GetOrder(ParseNode("[7,7,7,7]"), ParseNode("[7,7,7]")));
Assert.Equal(Order.LessThan, GetOrder(ParseNode("[]"), ParseNode("[3]")));
Assert.Equal(Order.GreaterThan, GetOrder(ParseNode("[[[]]]"), ParseNode("[[]]")));
Assert.Equal(Order.GreaterThan, GetOrder(ParseNode("[1,[2,[3,[4,[5,6,7]]]],8,9]"), ParseNode("[1,[2,[3,[4,[5,6,0]]]],8,9]")));

In [109]:
// Read input
var input = System.IO.File.ReadAllLines("input.txt");
input.Take(10)

index,value
0,"[[[5,1,[],[8,1,3],6],[[7]]],[10,[]],[6],[[],[[0,6,4,10,5],[2,2,9],[4,4],[2,10,4,10,8]],7,7],[[5],5,[[6],[8,2],[5],[]],[9,3,2,[9,4,8,6,8]]]]"
1,[[5]]
2,
3,"[[[5,3,[],[5,3,10],2],[[5,0,9,1]]],[[[9,0],[9,6,3]]],[[[1],1,[7,6],10],4],[2]]"
4,"[[],[[2,[9,2,6,9],10],[[2,8,10],[4,6],[1,6,4,6],[2,10,1,0,6]],6,[7,6,3,[10,6,4,5],4]],[],[2,2,[]],[2,9,[[3,3,9,6],[6,8,7,9,6],10,2,[6,9,8,7,0]],[7]]]"
5,
6,"[[[4,6,7]],[[1,10,8,[7,2]],10],[10],[[3,7,8,[6,7],[]],8,[4,[5,0,9,1],[6,1,2,6,5]]]]"
7,"[[[3,[1,8,7,6],[3,8,7,7],[],[8,5]],[[1,8,10],[10,8,9]],8,[[8],9,[],3],[7,9,8,[2,10,7]]],[8,2],[6]]"
8,
9,[[3]]


In [110]:
// Part 1
input.Chunk(3)
    .Select(lines => lines.ToList())
    .Select((lines, index) => GetOrder(ParseNode(lines[0]), ParseNode(lines[1])) == Order.LessThan ? index + 1 : 0)
    .Where(v => v != 0)
    .Sum()

5843

--- Part Two ---

Now, you just need to put all of the packets in the right order. Disregard the blank lines in your list of received packets.

The distress signal protocol also requires that you include two additional divider packets:
```
[[2]]
[[6]]
```
Using the same rules as before, organize all packets - the ones in your list of received packets as well as the two divider packets - into the correct order.

For the example above, the result of putting the packets in the correct order is:
```
[]
[[]]
[[[]]]
[1,1,3,1,1]
[1,1,5,1,1]
[[1],[2,3,4]]
[1,[2,[3,[4,[5,6,0]]]],8,9]
[1,[2,[3,[4,[5,6,7]]]],8,9]
[[1],4]
[[2]]
[3]
[[4,4],4,4]
[[4,4],4,4,4]
[[6]]
[7,7,7]
[7,7,7,7]
[[8,7,6]]
[9]
```
Afterward, locate the divider packets. To find the decoder key for this distress signal, you need to determine the indices of the two divider packets and multiply them together. (The first packet is at index 1, the second packet is at index 2, and so on.) In this example, the divider packets are 10th and 14th, and so the decoder key is 140.

Organize all of the packets into the correct order. What is the decoder key for the distress signal?


In [111]:
// Part 2
var orderedPackets = input.Where(line => String.IsNullOrEmpty(line) == false)
    .Append("[[2]]")
    .Append("[[6]]")
    .OrderBy(line => ParseNode(line), new NodeComparer())
    .ToList();

var firstPacketIndex = orderedPackets.IndexOf("[[2]]");
var secondPacketIndex = orderedPackets.IndexOf("[[6]]");
(firstPacketIndex + 1) * (secondPacketIndex + 1)

26289